In [71]:
using Random
using DataFrames
using CSV

struct Fighter
    name::String
    h::Int16
    f::Int16
    l::Int16
    t::Int16
end

const MXS = 400
const MXV = 40
const MNV = 2

function random_fighter(h0::Int = MNV, f0::Int = MNV, l0::Int = MNV, t0::Int = MNV)::Fighter
    flag = true
    while flag
        h = h0 + rand(0:(MXV - h0), 1)[1]
        f = f0 + rand(0:(MXV - f0), 1)[1]
        l = l0 + rand(0:(MXV - l0), 1)[1]
        t = t0 + rand(0:(MXV - t0), 1)[1]
        mm = minimum([h, f])
        budget = h*f+h*l+f*t
        if budget <= MXS && budget + mm > MXS
            flag = false
            return Fighter("No Name", h, f, l, t)
        end
    end
end

function eval_battle(a::Fighter, b::Fighter)::Int
    a_finds = a.f >= b.h
    b_finds = b.f >= a.h
    if a_finds && b_finds
        if a.t > b.t
            return 1
        end
        if a.t < b.t
            return -1
        end
        if a.t == b.t
            if a.l > b.l
                return 1
            end
            if a.l < b.l
                return -1
            end
            if a.l == b.l
                if a.h > b.h
                    return 1
                end
                if a.h < b.h
                    return -1
                end
                if a.h == b.h
                    return 0
                end
            end
        end
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        if a.l > b.l
            return 1
        end
        if a.l < b.l
            return -1
        end
        if a.l == b.l
            if a.t > b.t
                return 1
            end
            if a.t < b.t
                return -1
            end
            if a.t == b.t
                if a.f > b.f
                    return 1
                end
                if a.f < b.f
                    return -1
                end
                if a.f == b.f
                    return 0
                end
            end
        end
    end
end

function random_tournament_winner(c::Int)::Fighter
    if c==0
        return random_fighter()
    end
    a = random_tournament_winner(c-1)
    b = random_tournament_winner(c-1)
    res = eval_battle(a, b)
    if res == 1
        return a
    else
        return b
    end
end


tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)

function random_name_and_stat()::Fighter
    vp = [0, 0, 0, 0]
    nametype = rand(1:5)
    name = ""
    if nametype == 1 || nametype == 2
        nm = rand(names)
        adj_i = rand(1:nrow(adjectives))
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H]
        vp[2] = vp[2] + adjectives[adj_i, :F]
        vp[3] = vp[3] + adjectives[adj_i, :L]
        vp[4] = vp[4] + adjectives[adj_i, :T]
        if nametype == 1
            name = string(nm, " the ", adj)
        end
        if nametype == 2
            name = string(adj, " ", nm)
        end
    end
    if nametype == 3 || nametype == 4
        nm = rand(names)
        noun_i = rand(1:nrow(nouns))
        noun = nouns[noun_i, :noun]
        vp[1] = vp[1] + nouns[noun_i, :H]
        vp[2] = vp[2] + nouns[noun_i, :F]
        vp[3] = vp[3] + nouns[noun_i, :L]
        vp[4] = vp[4] + nouns[noun_i, :T]    
        if nametype == 3
            name = string(nm, " the ", noun)
        end
        if nametype == 4
            name = string(noun, " ", nm)
        end
    end
    if nametype == 5
        adj_i = rand(1:nrow(adjectives))
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H]
        vp[2] = vp[2] + adjectives[adj_i, :F]
        vp[3] = vp[3] + adjectives[adj_i, :L]
        vp[4] = vp[4] + adjectives[adj_i, :T]
        noun_i = rand(1:nrow(nouns))
        noun = nouns[noun_i, :noun]
        vp[1] = vp[1] + nouns[noun_i, :H]
        vp[2] = vp[2] + nouns[noun_i, :F]
        vp[3] = vp[3] + nouns[noun_i, :L]
        vp[4] = vp[4] + nouns[noun_i, :T]    
        name = string(adj, " ", noun)
    end
    Fighter(name, vp[1], vp[2], vp[3], vp[4])
end

function rand_rename(a::Fighter, n_tries::Int = 10)::Fighter
    best_score = 0.0
    best_b = random_name_and_stat()
    for ii in 1:n_tries
        b = random_name_and_stat()
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.t^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.t * b.t)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Fighter(best_b.name, a.h, a.f, a.l, a.t)
end

function eval_battle_list(a::Fighter, bs::Array{Fighter})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function pick_best(as::Array{Fighter}, bs::Array{Fighter})::Fighter
    bestscore = -999
    bestf = as[1]
    for ii in 1:length(as)
        score = eval_battle_list(as[ii], bs)
        if score > bestscore
            bestscore = score
            bestf = as[ii]
        end
    end
    return bestf
end

function pick_best_rdmly(as::Array{Fighter}, bs::Array{Fighter}, ntries::Int)::Fighter
    bestscore = -999
    bestf = rand(as)
    for ii in 1:ntries
        f = rand(as)
        score = eval_battle_list(f, bs)
        if score > bestscore
            bestscore = score
            bestf = f
        end
    end
    return bestf
end

function fighters_to_df(as::Array{Fighter})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ts = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ts[ii] = as[ii].t    
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, t = ts)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

fpart (generic function with 1 method)

In [12]:
function random_team(f::Function, team_size::Int)::Array{Fighter}
    team = Array{Fighter}(undef, team_size)
    for i in 1:team_size
        team[i] = f()
    end
    return team
end

random_team (generic function with 1 method)

In [2]:
library = Array{Fighter}(undef, 100000)
for i in 1:100000
    library[i] = rand_rename(random_fighter())
end

In [3]:
function evaluate_generator(f::Function, team_size::Int)::AbstractFloat
    team = Array{Fighter}(undef, team_size)
    for i in 1:team_size
        team[i] = f()
    end
    bestscore = 999999
    exploiter = library[1]
    for i in 1:length(library)
        fighter = library[i]
        score = 0
        for j in 1:team_size
            score = score + eval_battle(team[j], fighter)
        end
        if score < bestscore
            bestscore = score
            exploiter = library[i]
        end
    end
    print(exploiter)
    return bestscore/team_size
end

evaluate_generator (generic function with 1 method)

In [226]:
# new random generation to more closely approximate Nash
function random_scheme_a(budget::Int = MXS, h0::Int = MNV, f0::Int = MNV, l0::Int = MNV, t0::Int = MNV)::Fighter
    budget_sofar = budget + 1
    while budget_sofar > budget
        hf_ratio = rand()^(0.8)
        l_ratio = rand()
        h_temp = rand() + 0.5
        f_temp = rand() + 0.5
        budget_hf = (0.05 + hf_ratio * 0.9) * (budget * .7)
        h_ratio = h_temp/sqrt(h_temp * f_temp)
        f_ratio = f_temp/sqrt(h_temp * f_temp)
        h = max(h0, round(sqrt(budget_hf) * h_ratio))
        f = max(f0, round(sqrt(budget_hf) * f_ratio))
        budget_hf = h * f
        budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
        l_ratio2 = (l_ratio * h)/(l_ratio * h + (1-l_ratio) * f)
        lraw = l0 + ( budget_remain * l_ratio2 )/h
        traw = t0 + ( budget_remain * (1 - l_ratio2) )/f
        l = trunc(lraw)
        t = trunc(traw)
        budget_sofar = h * f + h * l + f * t
        if (budget - budget_sofar) >= min(h, f)
            could_add_l = (budget - budget_sofar) >= h
            could_add_t = (budget - budget_sofar) >= f
            if could_add_l && !could_add_t
                l = l+trunc((budget - budget_sofar)/h)
            end
            if !could_add_l && could_add_t
                t = t+trunc((budget - budget_sofar)/f)
            end
            if could_add_l && could_add_t
                if fpart(lraw) > fpart(traw)
                    l = l+trunc((budget - budget_sofar)/h)
                else
                    t = t+trunc((budget - budget_sofar)/f)
                end
            end
        end
    end
    h = convert(Int, h)
    f = convert(Int, f)
    l = convert(Int, l)
    t = convert(Int, t)  
    return Fighter("No Name [Scheme A]", h, f, l, t)
end



random_scheme_a (generic function with 6 methods)

In [227]:
budget = MXS
h0 = MNV
f0 = MNV
l0 = MNV
t0 = MNV
hf_ratio0 = 0
l_ratio0 = 0
h_temp0 = 0
f_temp0 = 0


for i in 1:1000
    hf_ratio = rand()^(0.8)
    l_ratio = 0.25 + 0.5 * rand()
    h_temp = rand() + 0.5
    f_temp = rand() + 0.5
    budget_hf = sqrt(0.05 + hf_ratio * 0.9) * (budget * .7)
    h_ratio = h_temp/sqrt(h_temp * f_temp)
    f_ratio = f_temp/sqrt(h_temp * f_temp)
    h = max(h0, round(sqrt(budget_hf) * h_ratio))
    f = max(f0, round(sqrt(budget_hf) * f_ratio))
    budget_hf = h * f
    budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
    lraw = l0 + ( budget_remain * l_ratio )/h
    traw = t0 + ( budget_remain * (1 - l_ratio) )/f
    l = trunc(lraw)
    t = trunc(traw)
    budget_sofar = h * f + h * l + f * t
    if (budget - budget_sofar) >= min(h, f)
        could_add_l = (budget - budget_sofar) >= h
        could_add_t = (budget - budget_sofar) >= f
        if could_add_l && !could_add_t
            l = l+trunc((budget - budget_sofar)/h)
        end
        if !could_add_l && could_add_t
            t = t+trunc((budget - budget_sofar)/f)
        end
        if could_add_l && could_add_t
            if fpart(lraw) > fpart(traw)
                l = l+trunc((budget - budget_sofar)/h)
            else
                t = t+trunc((budget - budget_sofar)/f)
            end
        end
    end

    budget_sofar = h * f + h * l + f * t
    if (budget - budget_sofar) >= min(h, f)
        hf_ratio0 = hf_ratio
        l_ratio0 = l_ratio
        h_temp0 = h_temp
        f_temp0 = f_temp
        println((h, f, l, t, budget_hf, budget_sofar))
    end
end

In [228]:
hf_ratio = hf_ratio0
l_ratio = l_ratio0
h_temp = h_temp0
f_temp = f_temp0

0

In [229]:
budget_hf = sqrt(0.05 + hf_ratio * 0.9) * (budget * .7)
h_ratio = h_temp/sqrt(h_temp * f_temp)
f_ratio = f_temp/sqrt(h_temp * f_temp)
h = max(h0, round(sqrt(budget_hf) * h_ratio))
f = max(f0, round(sqrt(budget_hf) * f_ratio))
budget_hf = h * f
budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
lraw = l0 + ( budget_remain * l_ratio )/h
traw = t0 + ( budget_remain * (1 - l_ratio) )/f
l = trunc(lraw)
t = trunc(traw)
budget_sofar = h * f + h * l + f * t
println((h, f, l, t, budget_hf, budget_sofar))

(NaN, NaN, NaN, NaN, NaN, NaN)


In [230]:
if (budget - budget_sofar) >= min(h, f)
    could_add_l = (budget - budget_sofar) >= h
    could_add_t = (budget - budget_sofar) >= f
    if could_add_l && !could_add_t
        l = l+1
    end
    if !could_add_l && could_add_t
        t = t+1
    end
    if could_add_l && could_add_t
        if fpart(lraw) > fpart(traw)
            l = l+1
        else
            t = t+1
        end
    end
end

budget_sofar = h * f + h * l + f * t
println((h, f, l, t, budget_hf, budget_sofar))

(NaN, NaN, NaN, NaN, NaN, NaN)


## Testing Random Scheme A

In [231]:
f = random_scheme_a()
print(f)
f.h * f.f + f.h * f.l + f.f * f.t

Fighter("No Name [Scheme A]", 21, 9, 4, 14)

399

In [232]:
rt = random_team(() -> rand_rename(random_scheme_a()), 100)
fighters_to_df(rt)[1:10, :]

Row,name,h,f,l,t
,String,Int64,Int64,Int64,Int64
1,Cheetah Nyelli,12,18,3,8
2,Balanced Alexi,14,17,6,4
3,Student Kennedy,12,18,12,2
4,Creative Yak,12,9,10,19
5,Idalis the Fish,19,12,7,3
6,Primordial Yak,20,10,2,16
7,Maggie the Lawyer,16,14,2,10
8,Sheltering Hava,14,9,12,11
9,Maddalynn the Rascal,23,11,3,7


In [233]:
counters = random_team(() -> pick_best_rdmly(library, rt, 9), 50)
fighters_to_df(counters)[1:10, :]

Row,name,h,f,l,t
,String,Int64,Int64,Int64,Int64
1,Duck Arch,27,2,11,24
2,Sheltering Luziano,22,2,14,24
3,Sword Kayani,4,16,16,17
4,Esperanza the Prankster,24,2,14,8
5,Balloon Odessa,16,8,11,12
6,Marilin the Koala,14,3,21,21
7,Jellyfish Shraga,14,13,11,4
8,Berklee the Quail,21,6,11,7
9,Giraffe Kayliani,3,20,6,16


In [234]:
for f in rt
    v = eval_battle_list(f, counters)
    if v < -30
        println(f)
        println(v)
    end
end

Fighter("Creative Yak", 12, 9, 10, 19)
-32
Fighter("Elephant Zay", 6, 8, 10, 36)
-32
Fighter("Tavaris the Yellow", 15, 11, 5, 14)
-32
Fighter("Frozen Icosahedron", 13, 11, 11, 10)
-32
Fighter("Ox Daphne", 9, 9, 7, 28)
-32
Fighter("Eliseo the Rascal", 14, 11, 8, 12)
-32
Fighter("Quiet Violet", 12, 13, 11, 8)
-38
Fighter("Kendryx the Super", 10, 9, 3, 31)
-32


In [235]:
evaluate_generator(random_fighter, 200)

Fighter("Yoyo Aleysa", 12, 11, 14, 9)

-0.47

In [238]:
evaluate_generator(random_scheme_a, 200)

Fighter("Tricky Lamp", 2, 23, 4, 15)

-0.64

In [239]:
evaluate_generator(() -> random_tournament_winner(1), 200)

Fighter("Georgia the Atmospheric", 18, 18, 2, 2)

-0.46

In [240]:
evaluate_generator(() -> random_tournament_winner(2), 200)

Fighter("Silent Ninja", 15, 17, 5, 4)

-0.47

In [241]:
evaluate_generator(() -> random_tournament_winner(5), 200)

Fighter("Dizzy Boot", 18, 2, 19, 11)

-0.53

In [242]:
evaluate_generator(() -> random_tournament_winner(6), 200)

Fighter("Cricket Keyomi", 22, 2, 16, 2)

-0.56